In [1]:
from lxml import html  
import json
import requests
import json,re
from dateutil import parser as dateparser
from time import sleep
import random
import pandas as pd
import os

## Helpers

In [ ]:
# tor handler class
# from here: https://github.com/bdheath/pytor
import stem
from stem.control import Controller
from stem import Signal
#import mechanize
import datetime
import requests

class pytor:
    controlPort = 9051
    port = 9050
    host = 'localhost'
    password = 'Password'
    _version = 0.1
    _last_result = None
    _last_request = None
    _id_time = 60
    _last_id_time = None
    _ip = None
    _connected = False
    browser = None
    controller = False
    _h = None
    _proxies = {}

    def __init__(self, host='localhost', port=9050, controller=True, controlPort = 9051):#, password=''):
        self.controlPort = controlPort
        self.port = port
        self.controller = controller
        self._proxies['http'] = 'socks5://localhost:' + str(self.port)
        self._proxies['https'] = 'socks5://localhost:' + str(self.port)
        if self.controller:
            self.torControl = Controller.from_port(port=controlPort)
            self.torControl.authenticate(self.password)
            self._connected = True
        self._last_id_time = datetime.datetime.now()
        return

    def request(self, url, h, timeout = 30):
        self._checkIdentityTime()
        r = requests.get(url, headers=h, timeout=timeout, proxies=self._proxies)
        print(f'GET through TOR the: {r.url}')
        self._last_request = url
        self._last_result = r.text
        return r

    def get(self, url, h=None, t=30):		
        self._checkIdentityTime()

        r = self.request(url, h, timeout = t)
        print(f'Status code: {r.status_code}')

        if r.status_code == 200:
            self._last_result = r.text
            self._last_request = url
            return self._last_result
        else:
            return False

    def ip(self):
        #self._ip = self.get('http://bradheath.org/ip', self._h)
        self._ip = self.get('https://api.ipify.org', self._h)
        return self._ip

    def timeForNew(self):
        self._checkIdentityTime()
        return		
        
    def saveLastResult(file):
        with open(file, 'wb') as local_file:
            local_file.write(self._last_result)
        return

    def downloadFile(self, url, file):
        self._checkIdentityTime()
        r = requests.get(url, stream = True, proxies = self._proxies)
        with open(file, 'wb') as local_file:
            for chunk in r.iter_content ( chunk_size = 1024):
                if chunk:
                    local_file.write(chunk)
        self._last_request = url
        return True

    def newIdentity(self):
        print("## NEW IDENTITY ## ")
        if not self._connected:
            self.torControl = Controller.from_port(port=self.controlPort)
            self.torControl.authenticate(self.password)
            self._connected = True
        self.torControl.authenticate(self.password)
        self.torControl.signal(Signal.NEWNYM)
        self._last_id_time = datetime.datetime.now()
        return True

    def _checkIdentityTime(self):
        if self._id_time != None and self._last_id_time != None:
            if (datetime.datetime.now() - self._last_id_time).seconds >= self._id_time:
                print('Getting new identity')
                self.newIdentity()
        return

    def identityTime(self, time = 600):
        self._id_time = time
        return

#	'''def mechanizeBrowser(self):
#		self.browser = mechanize.Browser()
#		return self.browser'''

In [ ]:
#headers generator
def getHeaders():
    user_agents = ['Mozilla/5.0 (Windows; U; Windows NT 5.1; en-US) AppleWebKit/525.19 (KHTML, like Gecko) Chrome/1.0.154.53 Safari/525.19','Mozilla/5.0 (Windows; U; Windows NT 5.1; en-US) AppleWebKit/525.19 (KHTML, like Gecko) Chrome/1.0.154.36 Safari/525.19','Mozilla/5.0 (Windows; U; Windows NT 6.1; en-US) AppleWebKit/534.10 (KHTML, like Gecko) Chrome/7.0.540.0 Safari/534.10','Mozilla/5.0 (Windows; U; Windows NT 5.2; en-US) AppleWebKit/534.4 (KHTML, like Gecko) Chrome/6.0.481.0 Safari/534.4','Mozilla/5.0 (Macintosh; U; Intel Mac OS X; en-US) AppleWebKit/533.4 (KHTML, like Gecko) Chrome/5.0.375.86 Safari/533.4','Mozilla/5.0 (Windows; U; Windows NT 5.1; en-US) AppleWebKit/532.2 (KHTML, like Gecko) Chrome/4.0.223.3 Safari/532.2','Mozilla/5.0 (Windows; U; Windows NT 6.1; en-US) AppleWebKit/532.0 (KHTML, like Gecko) Chrome/4.0.201.1 Safari/532.0','Mozilla/5.0 (Windows; U; Windows NT 5.2; en-US) AppleWebKit/532.0 (KHTML, like Gecko) Chrome/3.0.195.27 Safari/532.0','Mozilla/5.0 (Windows; U; Windows NT 5.1; en-US) AppleWebKit/530.5 (KHTML, like Gecko) Chrome/2.0.173.1 Safari/530.5','Mozilla/5.0 (Windows; U; Windows NT 5.2; en-US) AppleWebKit/534.10 (KHTML, like Gecko) Chrome/8.0.558.0 Safari/534.10','Mozilla/5.0 (X11; U; Linux x86_64; en-US) AppleWebKit/540.0 (KHTML,like Gecko) Chrome/9.1.0.0 Safari/540.0','Mozilla/5.0 (Windows; U; Windows NT 5.1; en-US) AppleWebKit/534.14 (KHTML, like Gecko) Chrome/9.0.600.0 Safari/534.14','Mozilla/5.0 (X11; U; Windows NT 6; en-US) AppleWebKit/534.12 (KHTML, like Gecko) Chrome/9.0.587.0 Safari/534.12','Mozilla/5.0 (Windows; U; Windows NT 6.1; en-US) AppleWebKit/534.13 (KHTML, like Gecko) Chrome/9.0.597.0 Safari/534.13','Mozilla/5.0 (Windows; U; Windows NT 6.1; en-US) AppleWebKit/534.16 (KHTML, like Gecko) Chrome/10.0.648.11 Safari/534.16','Mozilla/5.0 (Windows; U; Windows NT 6.0; en-US) AppleWebKit/534.20 (KHTML, like Gecko) Chrome/11.0.672.2 Safari/534.20','Mozilla/5.0 (Windows NT 6.0) AppleWebKit/535.1 (KHTML, like Gecko) Chrome/14.0.792.0 Safari/535.1','Mozilla/5.0 (Windows NT 5.1) AppleWebKit/535.2 (KHTML, like Gecko) Chrome/15.0.872.0 Safari/535.2','Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/535.7 (KHTML, like Gecko) Chrome/16.0.912.36 Safari/535.7','Mozilla/5.0 (Windows NT 6.0; WOW64) AppleWebKit/535.11 (KHTML, like Gecko) Chrome/17.0.963.66 Safari/535.11','Mozilla/5.0 (Macintosh; Intel Mac OS X 10_6_8) AppleWebKit/535.19 (KHTML, like Gecko) Chrome/18.0.1025.45 Safari/535.19','Mozilla/5.0 (Windows NT 6.2; WOW64) AppleWebKit/535.24 (KHTML, like Gecko) Chrome/19.0.1055.1 Safari/535.24','Mozilla/5.0 (Windows NT 6.2) AppleWebKit/536.6 (KHTML, like Gecko) Chrome/20.0.1090.0 Safari/536.6','Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.1 (KHTML, like Gecko) Chrome/22.0.1207.1 Safari/537.1','Mozilla/5.0 (Windows NT 6.2; WOW64) AppleWebKit/537.15 (KHTML, like Gecko) Chrome/24.0.1295.0 Safari/537.15','Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/27.0.1453.93 Safari/537.36','Mozilla/5.0 (Windows NT 6.2) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/28.0.1467.0 Safari/537.36','Mozilla/5.0 (Windows NT 6.3; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/30.0.1599.101 Safari/537.36','Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/31.0.1623.0 Safari/537.36','Mozilla/5.0 (Windows NT 6.2; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/34.0.1847.116 Safari/537.36','Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/37.0.2062.103 Safari/537.36','Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_2) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/40.0.2214.38 Safari/537.36','Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/46.0.2490.71 Safari/537.36','Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/51.0.2704.103 Safari/537.36','Mozilla/5.0 (Windows NT 6.1; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/62.0.3202.62 Safari/537.36']
    headers = {'User-Agent': user_agents[random.randint(0,len(user_agents)-1)]}
    return headers

#parser function
def getParser(url, headers, t=tor):
    #print(f'IP: {tor.ip()}')
    page = t.get(url,headers)
    
    if page:
        parser = html.fromstring(page)
        print(f'getParser: Returning {parser}')
        return parser
    else:
        print('getParser: Unable to parse page')
        return False

In [ ]:
#getting product info function
def getProductInfo(asin):
    
    #setup url to parse
    amazon_url  = 'http://www.amazon.com/dp/'+asin
        
    #obtain parsed page
    parser = getParser(amazon_url, getHeaders())
    
    if not parser:
        print("Has not received parser.")
        return False
    
    #set xpaths
    xpath_product_name     = '//h1//span[@id="productTitle"]//text()'
    xpath_product_price    = '//span[@id="priceblock_ourprice"]/text()'
    xpath_bullets          = '//div[@id="fbExpandableSectionContent"]//li/span/text()'
    xpath_description      = '//div[@id="productDescription"]//p/text()'
    xpath_description_2    = '//div[contains(@class,"aplus-module")]//p/text()'
    xpath_reviews_count    = '//span[@id="acrCustomerReviewText"]//text()'
    xpath_rating_count     = '//span[contains(@class,"reviewCountTextLinkedHistogram")]/@title'
    xpath_variation_flavor = '//div[@id="shelfSwatchSection-flavor_name"]//span[contains(@class,"twisterShelf_swatch_text")]/text()'
    xpath_variation_size   = '//div[@id="shelf-size_name"]//span[contains(@class,"twisterShelf_swatch_text")]/text()'
    xpath_seller_rank = '//li[@id="SalesRank"]/text()'
    xpath_main_category_url = '//li[@id="SalesRank"]/a/@href'
    
    #extract raw data
    raw_product_name          = parser.xpath(xpath_product_name)
    raw_product_price         = parser.xpath(xpath_product_price)
    raw_product_bullets       = parser.xpath(xpath_bullets)
    raw_product_description   = parser.xpath(xpath_description)
    raw_product_description_2 = parser.xpath(xpath_description_2)
    raw_reviews_count         = parser.xpath(xpath_reviews_count)
    raw_rating                = parser.xpath(xpath_rating_count)
    raw_variation_flavor      = parser.xpath(xpath_variation_flavor)
    raw_variation_size        = parser.xpath(xpath_variation_size)
    raw_seller_rank           = parser.xpath(xpath_seller_rank)
    raw_main_category         = parser.xpath(xpath_main_category)
    raw_main_category_url     = parser.xpath(xpath_main_category_url)
    
    #cleaning data
    product_name     = ''.join(raw_product_name).strip()
    product_price    = ''.join(raw_product_price).replace(',','').replace('$','')
    variation_flavor = ' '.join(','.join(raw_variation_flavor).split())
    variation_size   = ' '.join(','.join(raw_variation_size).split())
   
    if raw_product_description_2:
        product_description = ' '.join(' '.join(raw_product_description_2).split())
    else:
        product_description = ' '.join(' '.join(raw_product_description).split())

    product_bullets = {}
    for i in range(len(raw_product_bullets)):
        product_bullets['Bullet '+str(i+1)] = ' '.join(raw_product_bullets[i].split())
        
    if raw_reviews_count:
        reviews_count = ' '.join(raw_reviews_count).split()[0]
    else:
        reviews_count = 0
    
    if raw_rating:
        rating = ' '.join(raw_rating).split()[0]
    else:
        rating = None
        
    if raw_main_category:
        seller_rank   = ''.join(raw_seller_rank).split()[0].replace('#','').replace(',','')
        main_category = ' '.join(''.join(raw_seller_rank).split()[2:-1])
        main_category_url = raw_main_category_url[0]
    
    #create results dictionary
    results = {
        'ASIN'             :asin,
        'Name'             :product_name,
        'Price'            :product_price,
        'Flavours'         :variation_flavor,
        'Sizes'            :variation_size,
        'Rating'           :rating,
        'Reviews'          :reviews_count,
        'URL'              :amazon_url, 
        'Description'      :product_description,
        'Best Seller Rank' :seller_rank,
        'Main Category'    :main_category,
        'Main Category URL':main_category_url
        }
    
    #add product bullets so later DF will treat those as columns on the same line
    results.update(product_bullets)
    
    return results

In [ ]:
#getting reviews function
def getProductReviews(asin, pageNumber=1):    
    #setup resulting list
    reviews_list = []
    
    #setup url to parse
    review_url = 'https://www.amazon.com/product-reviews/'+asin+'?pageNumber='+str(pageNumber)+'&sortBy=recent'
    
    #obtain parsed page
    parser = getParser(review_url, getHeaders())
    
    #setup xpaths
    xpath_reviews           = '//div[@data-hook="review"]'
    xpath_no_review_section = '//div[contains(@class,"no-reviews-section")]//text()'
    
    #parse reviews
    reviews = parser.xpath(xpath_reviews)
    
    if not reviews:
        print('getProductReviews: Unable to find reviews in page')
        return reviews_list
    
    #parse end of reviews
    no_review = parser.xpath(xpath_no_review_section)
    if no_review:
        print('getProductReviews: End of reviews')
        return reviews_list

    #setup the rest of xpaths
    xpath_rating  = './/i[@data-hook="review-star-rating"]//text()' 
    xpath_title   = './/a[@data-hook="review-title"]//text()'
    xpath_author  = './/a[@data-hook="review-author"]//text()'
    xpath_date    = './/span[@data-hook="review-date"]//text()'
    xpath_body    = './/span[@data-hook="review-body"]//text()'
    xpath_helpful = './/span[@data-hook="helpful-vote-statement"]//text()'
    
    #process individual reviews
    for review in reviews:
        #DO review-comments section
        review_comments = {}
        
        #cleaning data
        rating  = ''.join(review.xpath(xpath_rating)).replace('out of 5 stars','')
        title   = ' '.join(' '.join(review.xpath(xpath_title)).split())
        author  = ' '.join(' '.join(review.xpath(xpath_author)).split())
        date    = review.xpath(xpath_date)
        body    = ' '.join(' '.join(review.xpath(xpath_body)).split())
        helpful = ' '.join(' '.join(review.xpath(xpath_helpful)).split())
        
        #convert date
        try:
            review_posted_date = dateparser.parse(''.join(date)).strftime('%d %b %Y')
        except:
            review_posted_date = date
        
        review_dict = {'rating' : rating,
                   'title'  : title,
                   'author' : author,             
                   'date'   : review_posted_date,
                   'body'   : body,
                   'helpful': helpful,
                   'ASIN'   : asin
                    }
        review_dict.update(review_comments)
        reviews_list.append(review_dict)
    
    return reviews_list

## List of ASINs 

In [ ]:
AsinList = ['B016OP6N3M','B0723FXC74','B002M782UO','B01HN7ZGUQ','B000CBOTQ8',
            'B014HPPAFS','B01GP2MTXW','B000AAM0EY','B000AAM0G2','B01LZFRZWH']

## Setting TOR object

In [ ]:
#needed for switching identities
#needed to switch identities every 10 minutes
tor = pytor()

## Collecting product info

In [ ]:
#collecting product info
def ReadAsins(AsinList):
    #list of columns for resulting CSV file
    column_names = ['ASIN','URL','Name','Price','Flavours','Sizes','Rating','Reviews','Description',
     'Bullet 1','Bullet 2','Bullet 3','Bullet 4',
     'Bullet 5','Bullet 6','Bullet 7','Bullet 8']
    
    for asin in AsinList:
        print("Downloading and processing "+asin)
        
        results = getProductInfo(asin)
        print(results)

        if results:
            df = pd.DataFrame(results, index=[0], columns=column_names)

            # if file does not exist write header 
            if not os.path.isfile('products.csv'):
                df.to_csv('products.csv',index=False, )
            else: # else it exists so append without writing the header
                df.to_csv('products.csv',mode = 'a',header=False,index=False, columns=column_names)
        #sleep(random.uniform(5,15))
        print('Next')

if __name__ == '__main__':
	ReadAsins(AsinList)
    
print('Finished')

## Collecting reviews

In [ ]:
#collecting product reviews
def ReadAsins(AsinList):
    #list of columns for resulting CSV file
    column_names = ['ASIN','date','author','title','rating','body','helpful']   
    processed_asins = {}
    for asin in AsinList:
        no_more_pages = False
        pageNumber = 1 
        while no_more_pages == False:
            processed_asins[asin] = {'Latest reviews page processed':pageNumber}
            print("Downloading and processing reviews for "+asin)

            results = getProductReviews(asin, pageNumber)
            print(results)

            if results:
                df = pd.DataFrame(results, columns=column_names)
                
                # if file does not exist write with header 
                if not os.path.isfile('reviews.csv'):
                    df.to_csv('reviews.csv',index=False, )
                else: # else it exists so append without writing the header
                    df.to_csv('reviews.csv',mode = 'a',header=False,index=False, columns=column_names)
            else:
                no_more_pages = True

            #try next page
            pageNumber += 1
            sleep(random.uniform(1,3))
            print('Next')
            
        processed_df = pd.DataFrame(processed_asins, index=[0])
        if not os.path.isfile('processed.csv'):
            df.to_csv('processed.csv',index=False, )
        else: # else it exists so append without writing the header
            df.to_csv('processed.csv',mode = 'a',header=False,index=False)
    
            

if __name__ == '__main__':
    ReadAsins(AsinList)
    
print('Finished')

## Sandbox: